<a href="https://colab.research.google.com/github/kridtapon/ADX-R-Swing-Sync/blob/main/ADX_R_Swing_Sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [ ]:
pip install --upgrade yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# === Define custom indicators ===

def calculate_williams_r(df, period=14):
    highest_high = df['High'].rolling(window=period).max()
    lowest_low = df['Low'].rolling(window=period).min()
    williams_r = -100 * (highest_high - df['Close']) / (highest_high - lowest_low)
    return williams_r

def calculate_adx_adxr(df, period=14):
    high = df['High']
    low = df['Low']
    close = df['Close']

    # Calculate True Range
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # Calculate +DM and -DM
    plus_dm = (high - high.shift(1)).where((high - high.shift(1)) > (low.shift(1) - low), 0).clip(lower=0)
    minus_dm = (low.shift(1) - low).where((low.shift(1) - low) > (high - high.shift(1)), 0).clip(lower=0)

    # Smooth TR, +DM, -DM
    atr = tr.rolling(window=period).mean()
    plus_di = 100 * plus_dm.rolling(window=period).mean() / atr
    minus_di = 100 * minus_dm.rolling(window=period).mean() / atr

    # Calculate DX and ADX
    dx = (100 * (plus_di - minus_di).abs() / (plus_di + minus_di)).fillna(0)
    adx = dx.rolling(window=period).mean()

    # ADXR: average of current ADX and ADX from `period` bars ago
    adxr = (adx + adx.shift(period)) / 2
    return adx, adxr

# === Download Data ===
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'
df = yf.download(symbol, start=start_date, end=end_date, multi_level_index=False)

# === Calculate Indicators ===
df['Williams_%R'] = calculate_williams_r(df, period=14)
df['ADX'], df['ADXR'] = calculate_adx_adxr(df, period=14)

# === Define Entry and Exit Signals ===
df['W%R_prev'] = df['Williams_%R'].shift(1)

# Entry: ADXR > 25 and Williams %R crosses above -80
df['Entry'] = (
    (df['ADXR'] > 25) &
    (df['W%R_prev'] < -80) & (df['Williams_%R'] >= -80)
)

# Exit: ADXR > 25 and Williams %R crosses below -20
df['Exit'] = (
    (df['ADXR'] > 25) &
    (df['W%R_prev'] > -20) & (df['Williams_%R'] <= -20)
)

# === Filter and Prepare Signals ===
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
entries = df['Entry'].shift(1).astype(bool).fillna(False).to_numpy()
exits = df['Exit'].shift(1).astype(bool).fillna(False).to_numpy()

# === Run Portfolio Simulation ===
portfolio = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='D'
)

# === Output Results ===
print(portfolio.stats())
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Start                                2020-01-02 00:00:00
End                                  2024-12-31 00:00:00
Period                                1258 days 00:00:00
Start Value                                     100000.0
End Value                                  118495.628095
Total Return [%]                               18.495628
Benchmark Return [%]                          187.561082
Max Gross Exposure [%]                             100.0
Total Fees Paid                              5529.437629
Max Drawdown [%]                               65.833139
Max Drawdown Duration                  862 days 00:00:00
Total Trades                                          30
Total Closed Trades                                   29
Total Open Trades                                      1
Open Trade PnL                              -2452.723532
Win Rate [%]                                   65.517241
Best Trade [%]                                 32.965214
Worst Trade [%]                